In [2]:
import pandas as pd
import pyarrow
import pyreadr

In [17]:
d1 = pyreadr.read_r('../Daten/pw_lists/Badische Zeitung_links.RDS')
#d2 = pyreadr.read_r('../Daten/pw_lists/Berliner Morgenpost_links.RDS')
d2 = pyreadr.read_r('../Daten/pw_lists/Berliner Zeitung_links.RDS')
d3 = pyreadr.read_r('../Daten/pw_lists/Bild_links.RDS')
d4 = pyreadr.read_r('../Daten/pw_lists/Compact_links.RDS')
d5 = pyreadr.read_r('../Daten/pw_lists/faz_links.RDS')
d6 = pyreadr.read_r('../Daten/pw_lists/Hamburger MoPo_links.RDS')
d7 = pyreadr.read_r('../Daten/pw_lists/Handelsblatt_links.RDS')
d8 = pyreadr.read_r('../Daten/pw_lists/Jacobin_links.RDS')
d9 = pyreadr.read_r('../Daten/pw_lists/Junge Freiheit_links.RDS')
#d11 = pyreadr.read_r('../Daten/pw_lists/Junge Welt_links.RDS')
d10 = pyreadr.read_r('../Daten/pw_lists/Jungle World_links.RDS')
d11 = pyreadr.read_r('../Daten/pw_lists/KN_links.RDS')
d12 = pyreadr.read_r('../Daten/pw_lists/LVZ_links.RDS')
#d15 = pyreadr.read_r('../Daten/pw_lists/nordbayern.de_links.RDS_links.RDS')
d13 = pyreadr.read_r('../Daten/pw_lists/Ostsee-Zeitung_links.RDS')
d14 = pyreadr.read_r('../Daten/pw_lists/RND_links.RDS')
d15 = pyreadr.read_r('../Daten/pw_lists/RP Online_links.RDS')
d16 = pyreadr.read_r('../Daten/pw_lists/Spiegel_links.RDS')
d17 = pyreadr.read_r('../Daten/pw_lists/Stuttgarter Zeitung_links.RDS')
d18 = pyreadr.read_r('../Daten/pw_lists/SZ_links.RDS')
#d22 = pyreadr.read_r('../Daten/pw_lists/TA_links.RDS')
d19 = pyreadr.read_r('../Daten/pw_lists/Tagesspiegel_links.RDS')
d20 = pyreadr.read_r('../Daten/pw_lists/Welt_links.RDS')

d21 = pd.read_csv('../Daten/pw_lists/BM_file.csv')
d22 = pd.read_csv('../Daten/pw_lists/JW_file.csv')
d23 = pd.read_csv('../Daten/pw_lists/nb_file.csv')
d24 = pd.read_csv('../Daten/pw_lists/TA.csv')


In [32]:
dataframes = [globals()[f'd{i}'] for i in range(1, 21)]
modified_dataframes = [df[None] for df in dataframes]
final_dataframe = pd.concat(modified_dataframes, ignore_index=True)


In [37]:
fd = final_dataframe[final_dataframe['paywalled'] == True]


### remove whole rows

In [413]:
filtered = JWo[JWo['text'].str.contains(r'Noch kein Abonnement', case=False, regex=True)]
filtered.shape

(244, 5)

In [414]:
ids_to_remove = filtered['_id'].unique().tolist()
JWo = JWo[~JWo['_id'].isin(ids_to_remove)]

### Count rows containig certain phrases

In [415]:
pattern = r'Um diesen Inhalt zu lesen, wird ein Online-Abo benötigt' 
count = sum(JWo['text'].str.contains(pattern, regex=True, flags=re.IGNORECASE))

print("Number of rows mentioning 'ads':", count)


Number of rows mentioning 'ads': 0


### remove sections (before a certain phrase)

In [10]:
def remove_before_phrase(text, phrase='THEMEN'):
    """
    Removes everything before and including a specific phrase in a text.

    Parameters:
    - text (str): The text to process.
    - phrase (str): The phrase to search for. Everything before and including this phrase will be removed.

    Returns:
    - str: The processed text with the specified content removed.
    """
    pos = text.find(phrase)
    if pos != -1:
        return text[pos + len(phrase):].lstrip()
    else:
        return text

In [ ]:
Focus['text'] = Focus['text'].apply(lambda x: remove_before_phrase(x))

### remove sections (after a certain phrase)

In [397]:
def remove_topics_and_following(text):
    """
    Removes the row that contains only 'topics' (case-insensitive) and everything that follows.

    Parameters:
    - text (str): The text to process.

    Returns:
    - str: The modified text with 'topics' and subsequent content removed.
    """
    lines = text.split('\n')
    
    for i, line in enumerate(lines):
        if line.strip().lower() == 'Die Regierung will nicht, dass Sie das hier über die Euro-Situation lesen:':
            return '\n'.join(lines[:i])
    
    return text


In [398]:

NP['text'] = NP['text'].apply(remove_topics_and_following)

### remove certain rows

In [427]:
def remove_specific_lines(text, phrase='Weiterlesen nach der Anzeige'):
    """
    Removes lines from a text that start with a specific phrase.

    Parameters:
    - text (str): The text to process.
    - phrase (str): The phrase that, if a line starts with, will cause the line to be removed.

    Returns:
    - str: The processed text with specific lines removed.
    """
    lines = text.split('\n')
    filtered_lines = [line for line in lines if not line.lower().startswith(phrase.lower())]
    return '\n'.join(filtered_lines)


In [428]:
KN['text'] = KN['text'].apply(remove_specific_lines)

In [326]:
def remove_length_of_article(text):
    """
    Removes the section starting with 'length of article:' up to the next double newline,
    accounting for variations in how the text is structured.

    Parameters:
    - text (str): The text to process.

    Returns:
    - str: The text with the specified sections removed.
    """
    pattern = re.compile(r'(?i)\s*NRW\s*.*?\n\n', flags=re.DOTALL)
    return pattern.sub('', text)


In [357]:
def remove_length_of_article(text):
    """
    Removes the section starting with 'length of article:' up to the next double newline,
    accounting for variations in how the text is structured.

    Parameters:
    - text (str): The text to process.

    Returns:
    - str: The text with the specified sections removed.
    """
    pattern = re.compile(r'(?i)\s*1 \/ 1\s*.*?slide 1 of 1', flags=re.DOTALL)
    return pattern.sub('', text)

In [358]:
Weser['text'] = Weser['text'].apply(remove_length_of_article)

### remove datetime

In [328]:
def remove_date_time_rows(text):
    """
    Removes rows that contain only a date and time in the format 'dd.mm.yyyy, hh:mm Uhr'.
    
    Parameters:
    - text (str): The text to process.

    Returns:
    - str: The text with date and time rows removed.
    """
    pattern = re.compile(r'^\d{1,2}\.\d{1,2}\.\d{4}, \d{1,2}:\d{2} Uhr', flags=re.MULTILINE)
    
    filtered_lines = [line for line in text.split('\n') if not pattern.match(line)]
    return '\n'.join(filtered_lines)

# r'^\d{2}\.\d{2}\.\d{4}, \d{2}:\d{2} Uhr$'


In [355]:
WAZ['text'] = WAZ['text'].apply(remove_date_time_rows)

In [354]:
def remove_date_time_rows(text):
    """
    Removes rows that contain only a date and time in the format 'dd.mm.yyyy, hh:mm Uhr'.
    
    Parameters:
    - text (str): The text to process.

    Returns:
    - str: The text with date and time rows removed.
    """
    pattern = re.compile(r'^\d{1,2}\.\d{1,2}\.\d{4}, \d{1,2}:\d{2}', flags=re.MULTILINE)
    
    filtered_lines = [line for line in text.split('\n') if not pattern.match(line)]
    return '\n'.join(filtered_lines)